In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import csv
import re
!pip install cltk
import cltk
# from cltk.corpus.utils.importer import CorpusImporter
from cltk.tokenize.word import WordTokenizer
from cltk.tag.pos import POSTag
from cltk.lemmatize.backoff import BackoffGreekLemmatizer

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.8/849.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ModuleNotFoundError: No module named 'cltk.corpus'

In [ ]:
workdir = '/content/drive/MyDrive/FactGrid Cuneiform (AWCA)/people/Tina/' #Tina

In [ ]:
test_df = pd.read_csv(f'{workdir}/test_words.csv')
test_df = test_df.fillna(0)
test_df['sentence'] = pd.to_numeric(test_df['sentence'], errors='coerce').fillna(0)
test_df

,Unnamed: 0,identifier,word,sentence,clause (;),footnotes,page
0,0,0.0.0,46,0.0,0.0,0.0,46.0
1,1,0.0.1,Pascal,0.0,0.0,0.0,0.0
2,2,0.0.2,Attinger,0.0,0.0,0.0,0.0
3,3,0.0.3,A,0.0,0.0,0.0,0.0
4,4,0.0.4,propos,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
11237,11237,0.18.707,la-ba-an-ak-,0.0,0.0,0.0,0.0
11238,11238,0.18.708,e,0.0,0.0,0.0,0.0
11239,11239,0.18.709,.,0.0,0.0,0.0,0.0
11240,11240,0.18.710,à,0.0,0.0,0.0,0.0


In [ ]:
test_df.loc[test_df['sentence'] == 1]

,Unnamed: 0,identifier,word,sentence,clause (;),footnotes,page
23,23,0.0.23,«faire».,1.0,0.0,0.0,0.0
71,71,0.0.71,(5).,1.0,0.0,0.0,0.0
92,92,0.0.92,(6).,1.0,0.0,0.0,0.0
134,134,0.0.134,70–131).,1.0,0.0,0.0,0.0
209,209,0.0.209,Museum.,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5561,5561,0.9.620,{e-pe}-sˇum).,1.0,0.0,0.0,0.0
5575,5575,0.9.634,signe]).,1.0,0.0,0.0,0.0
5591,5591,0.9.650,291).,1.0,0.0,0.0,0.0
5605,5605,0.9.664,3,1.0,0.0,0.0,0.0


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Example values for necessary variables
vocab_size = 1000  # Adjust based on your vocabulary size
embedding_dim = 100  # Embedding dimensions
units = 50  # Number of units in LSTM layer
epochs = 10  # Number of training epochs

# Tokenization and padding (assuming your data is in 'X' as text)
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(sequences, maxlen=100)  # maxlen should be chosen based on your data

# Splitting the data (assuming 'y' is your labels)
X_train_padded, X_test_padded, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Building the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(Bidirectional(LSTM(units)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train, epochs=epochs, class_weight={0: 1, 1: 10}, validation_split=0.1)  # Adjust class_weight based on your class distribution

# Predict and evaluate
y_pred = (model.predict(X_test_padded) > 0.5).astype('int32')
print(classification_report(y_test, y_pred))


In [ ]:
x_pred = rf_model.predict(X_train)
pred = np.concatenate((x_pred.flatten(), y_pred.flatten()))
test_df['predict'] = pred
test_df.loc[test_df['predict']== 1]
test_df.loc[test_df['sentence']== 1]

,Unnamed: 0,identifier,word,sentence,clause (;),footnotes,page,predict,predict_0_1
23,23,0.0.23,«faire».,1.0,0.0,0.0,0.0,0.0,0
71,71,0.0.71,(5).,1.0,0.0,0.0,0.0,0.0,0
92,92,0.0.92,(6).,1.0,0.0,0.0,0.0,1.0,0
134,134,0.0.134,70–131).,1.0,0.0,0.0,0.0,0.0,0
209,209,0.0.209,Museum.,1.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
5561,5561,0.9.620,{e-pe}-sˇum).,1.0,0.0,0.0,0.0,0.0,0
5575,5575,0.9.634,signe]).,1.0,0.0,0.0,0.0,0.0,0
5591,5591,0.9.650,291).,1.0,0.0,0.0,0.0,0.0,0
5605,5605,0.9.664,3,1.0,0.0,0.0,0.0,0.0,0


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load your dataset
df = test_df

# Prepare text data and labels
words = df['word'].astype(str).tolist()
labels = df['sentence'].fillna(0).tolist()


# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
sequences = tokenizer.texts_to_sequences(words)

# Padding sequences to ensure uniform length
max_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_length, padding='post')

# Convert labels to numpy array
labels = np.array(labels)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(tokenizer.word_index) + 1  # Plus one for padding token
embedding_dim = 100  # Adjust as needed
lstm_units = 50  # Adjust as needed

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(lstm_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
from sklearn.model_selection import train_test_split

# Assuming 'data' and 'labels' are your full dataset and labels
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Now, use 'train_data' and 'train_labels' for training
model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# And 'test_data' and 'test_labels' for evaluation
loss, accuracy = model.evaluate(test_data, test_labels)
train_pred = model.predict(train_data)
test_pred = model.predict(test_data)

Epoch 1/10
225/225 [==============================] - 8s 21ms/step - loss: 0.1180 - accuracy: 0.9862 - val_loss: 0.0586 - val_accuracy: 0.9894
Epoch 2/10
225/225 [==============================] - 2s 11ms/step - loss: 0.0663 - accuracy: 0.9862 - val_loss: 0.0570 - val_accuracy: 0.9894
Epoch 3/10
225/225 [==============================] - 3s 12ms/step - loss: 0.0515 - accuracy: 0.9851 - val_loss: 0.0658 - val_accuracy: 0.9878
Epoch 4/10
225/225 [==============================] - 3s 12ms/step - loss: 0.0427 - accuracy: 0.9885 - val_loss: 0.0657 - val_accuracy: 0.9872
Epoch 5/10
225/225 [==============================] - 3s 15ms/step - loss: 0.0403 - accuracy: 0.9889 - val_loss: 0.0614 - val_accuracy: 0.9878
Epoch 6/10
225/225 [==============================] - 4s 16ms/step - loss: 0.0394 - accuracy: 0.9893 - val_loss: 0.0619 - val_accuracy: 0.9883
Epoch 7/10
225/225 [==============================] - 3s 12ms/step - loss: 0.0396 - accuracy: 0.9890 - val_loss: 0.0694 - val_accuracy: 0.9867

In [ ]:
predicted_label = np.concatenate((train_pred.flatten(), test_pred.flatten()))
class_predictions = (predicted_label >= 0.99).astype(int)
class_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_df['predict'] = predicted_label
test_df['predict_0_1'] = class_predictions
test_df.loc[test_df['predict_0_1'] == 1]
test_df

,Unnamed: 0,identifier,word,sentence,clause (;),footnotes,page,predict,predict_0_1
0,0,0.0.0,46,0.0,0.0,0.0,46.0,0.000330,0
1,1,0.0.1,Pascal,0.0,0.0,0.0,0.0,0.014557,0
2,2,0.0.2,Attinger,0.0,0.0,0.0,0.0,0.000275,0
3,3,0.0.3,A,0.0,0.0,0.0,0.0,0.000638,0
4,4,0.0.4,propos,0.0,0.0,0.0,0.0,0.000260,0
...,...,...,...,...,...,...,...,...,...
11237,11237,0.18.707,la-ba-an-ak-,0.0,0.0,0.0,0.0,0.002615,0
11238,11238,0.18.708,e,0.0,0.0,0.0,0.0,0.000285,0
11239,11239,0.18.709,.,0.0,0.0,0.0,0.0,0.000529,0
11240,11240,0.18.710,à,0.0,0.0,0.0,0.0,0.000609,0


In [ ]:
from sklearn.metrics import accuracy_score

# Convert 'predict' column to numpy array for efficiency
predict_probs = np.array(test_df['predict'])

# Real labels
real_labels = test_df['sentence']


# Test thresholds
thresholds = np.arange(0, 1.01, 0.01)
best_accuracy = 0
best_threshold = 0

for threshold in thresholds:
    # Classify as 1 if 'predict' is greater than or equal to threshold, else 0
    predicted_labels = (predict_probs >= threshold).astype(int)

    # Calculate accuracy
    accuracy = accuracy_score(real_labels, predicted_labels)

    # Update best threshold if this threshold gives better accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

# Output the best threshold and corresponding accuracy
best_threshold, best_accuracy

(0.9, 0.9866571784380004)

In [ ]:
sentences = []
current_sentence = ''
start_identifier = None

for index, row in test_df.iterrows():
    if start_identifier is None:
        start_identifier = row['identifier']

    current_sentence += row['word'] + ' '


    if row['sentence'] == 1:

        sentences.append({
            'identifier': f"{start_identifier}-{row['identifier']}",
            'sentence': current_sentence.strip()
        })

        current_sentence = ''
        start_identifier = None

sentences_df = pd.DataFrame(sentences)


In [ ]:
sentences_df

,identifier,sentence
0,0.0.0-0.0.23,46 Pascal Attinger A propos de AK «faire»1 (I)...
1,0.0.24-0.0.71,Après une discussion du para- digme -2 et des ...
2,0.0.72-0.0.92,Des remarques de détail touchant les autres le...
3,0.0.93-0.0.134,1 Introduction Il est bien connu qu’à côté de ...
4,0.0.135-0.0.209,Il va sans dire que ces pages rendront d’inest...
...,...,...
145,0.9.611-0.9.620,ppB lex. a #ERROR! {ak}: Ea VIII 23 (= {e-pe}-...
146,0.9.621-0.9.634,( a-) ak #ERROR! {ak}: SbB 291 (= e-pe-[sˇu] /...
147,0.9.635-0.9.650,a-[x] #ERROR! {ak}: Ea VIII 25 (= MIN ({e-pe}-...
148,0.9.651-0.9.664,"22 Prov. inc. (pas de Nippur!); cf. M. Civil, ..."


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = 'ahxt/LiteLlama-460M-1T'

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

prompt = 'Q: Can you tell us what language is it:xlv + 217 Je me dois de remercier en premier lieu W. Sallaberger, qui a lu une version antérieure de cet article et dont les avis autorisés m’ont été un inestimable appui lors de la rédac- tion souvent douloureuse de cet article.	J’ai eu accès aux textes sur disquettes de B. Ja- gersma (époque présargonique) et de R. de Maaijer (époques d’Ur III et d’Isin).	Que leurs auteurs trouvent ici l’expression de ma reconnaissance.'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
tokens = model.generate(input_ids, max_length=400)
print( tokenizer.decode(tokens[0].tolist(), skip_special_tokens=True) )
# Q: What is the largest bird?\nA: The largest bird is a black-headed gull.

Q: Can you tell us what language is it:xlv + 217 Je me dois de remercier en premier lieu W. Sallaberger, qui a lu une version antérieure de cet article et dont les avis autorisés m’ont été un inestimable appui lors de la rédac- tion souvent douloureuse de cet article.	J’ai eu accès aux textes sur disquettes de B. Ja- gersma (époque présargonique) et de R. de Maaijer (époques d’Ur III et d’Isin).	Que leurs auteurs trouvent ici l’expression de ma reconnaissance.	Je ne suis pas un homme de lettres, mais je suis un homme de la littérature.	Je ne suis pas un homme de la littérature, mais je suis un homme de la littérature.	Je ne suis pas un homme de la littérature, mais je suis un homme de la littérature.	Je ne suis pas un homme de la littérature, mais je suis un homme de la littérature.	Je ne suis pas un homme de la littérature, mais je suis un homme de la littérature.	Je ne suis pas un homme de la littérature, mais je suis un homme de la littérature.	Je ne suis pas un homme de la littérat

Second data set test

In [ ]:
test_df_2 = pd.read_csv('/content/drive/MyDrive/FactGrid Cuneiform (AWCA)/people/Tina/1_Akanuma-IronKaman(2002)/words.csv')
test_df_2

,Unnamed: 0,identifier,word
0,0,1.0.0,ISSN
1,1,1.0.1,1345-7829
2,2,1.0.2,Anatolian
3,3,1.0.3,Archaeological
4,4,1.0.4,Studies
...,...,...,...
5997,5997,1.7.391,"city,"
5998,5998,1.7.392,Iwate
5999,5999,1.7.393,020-0102
6000,6000,1.7.394,JAPAN


In [ ]:
# Assuming test_df_2 is your second dataset
words_test_2 = test_df_2['word'].astype(str).tolist()

# Tokenize and pad the second dataset using the same tokenizer and max_length
sequences_test_2 = tokenizer.texts_to_sequences(words_test_2)
data_test_2 = pad_sequences(sequences_test_2, maxlen=max_length, padding='post')
# max_length = max(len(seq) for seq in sequences_test_2)
# Now you can predict using the fitted model
predictions = model.predict(data_test_2)

# The predictions variable now contains the output from your model
class_predictions = (predictions >= 0.5).astype(int)

188/188 [==============================] - 1s 3ms/step


In [ ]:
test_df_2['predict'] = class_predictions
test_df_2

,Unnamed: 0,identifier,word,pred_label,predict
0,0,1.0.0,ISSN,0,0
1,1,1.0.1,1345-7829,0,0
2,2,1.0.2,Anatolian,0,0
3,3,1.0.3,Archaeological,0,0
4,4,1.0.4,Studies,0,0
...,...,...,...,...,...
5997,5997,1.7.391,"city,",0,0
5998,5998,1.7.392,Iwate,0,0
5999,5999,1.7.393,020-0102,0,0
6000,6000,1.7.394,JAPAN,0,0


In [ ]:
sentences = []
current_sentence = ''
start_identifier = None

for index, row in test_df_2.iterrows():
    if start_identifier is None:
        start_identifier = row['identifier']

    current_sentence += row['word'] + ' '


    if row['predict'] == 1:

        sentences.append({
            'identifier': f"{start_identifier}-{row['identifier']}",
            'predict': current_sentence.strip()
        })

        current_sentence = ''
        start_identifier = None

sentences_df = pd.DataFrame(sentences)

In [ ]:
sentences_df

,identifier,predict
0,1.0.0-1.3.367,ISSN 1345-7829 Anatolian Archaeological Studie...
1,1.3.368-1.3.376,"|WaS 21 [63.33 0.054 0.005 0,003 <0.001 0.01 _..."
2,1.3.377-1.3.444,0.052 _0.003_— = - = = = (Cm(0.2~0.3)|no 22 [5...


In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

wnl = WordNetLemmatizer()

""" Generator-based lemmatizer for large Enwik9 chunks.

Might take a long time to run, but should not crash due to
memory error.

asahala 2023 """

def lemmatize_data(text):
    """ Lemmatize untokenized text using NLTK
    :param text      input text
    :type text       str """

    d = []
    itot = len(word_tokenize(text))
    i = 0
    for word, tag in pos_tag(word_tokenize(text)):

        if i % 1000 == 0:
            print(i, itot)

        wntag = tag[0].lower()
        wntag = wntag if wntag in ('a', 'r', 'n', 'v') else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)

        d.append(lemma)

        i += 1

    return d


def lemmatize_large_data(text, tokens_per_cycle=50000, output_size=2000000):
    """ Lemmatize untokenized text using NLTK

    :param text      input text
    :type text       stt

    Because NLTK is slow for large data sets, this will lemmatize
    50000 tokens at a time and write the output in files of 2 million
    tokens each that can be catenated later.

    """

    #d = []
    #itot = len(word_tokenize(text))
    #i = 0

    chunk = []
    part = 0
    for e, w in enumerate(text, start=1):

        chunk.append(w)
        if e % tokens_per_cycle == 0:
            with open(f'chunks/enwik9-2M-{str(part).zfill(3)}.txt', 'a', encoding='utf-8') as output:
                print(f'Processing chunk {e}')
                for word, tag in pos_tag(chunk):
                    wntag = tag[0].lower()
                    wntag = wntag if wntag in ('a', 'r', 'n', 'v') else None
                    if not wntag:
                        lemma = word
                    else:
                        lemma = wnl.lemmatize(word, wntag)
                    output.write(lemma + '\n')
                #print(f'Written to output {len(chunk)}')
                chunk = []
                #output.write('#' + '\n')

        if e % output_size == 0:
            #output.write('<EOF>\n')
            part += 1


def parse_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f.read().splitlines():
            yield line

def parse_large_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            yield line[:-1]

def write_data(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        for line in data:
            f.write(line + '\n')
        #f.write('#\n')

text = parse_large_data(f'{workdir}/test.txt')
lemmatize_large_data(text)



print('parsing')
text = ' '.join(list(parse_data(f'{workdir}/test.txt')))
print('lemmatizing')
data = lemmatize_data(text)
print('writing')
# write_data('data/enwik9-40M-part1-lem.txt', data)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


parsing
lemmatizing
0 741
writing


In [ ]:
text

'\ufeffIRON OBJECTS FROM THE ARCHITECTURAL REMAINS OF STRATUM III AND STRATUM II AT KAMAN-KALEHOYUK: CORRELATION BETWEEN COMPOSITION AND ARCHAEOLOGICAL LEVELS   Hideo AKANUMA   1 INTRODUCTION   During excavations from 1986 to 2001 at the site of Kaman-Kalehdyiik in central Anatolia, Turkey, many iron objects have been found. Archaeological and archaeometallurgical analysis have revealed that iron use at the site began in the cultural age of Stratum IIIc, in the Assyrian Colony Period\' (Akanuma 1997), and that steel production and the manufacture of iron artifacts were probably carried out during the Hittite Empire Period there (Akanuma 1995). A progressive increase in the deposits of iron artifacts, lumps, and slag was seen in the excavation layers of Stratum I] (Akanuma 1999). This progression, which becomes more evident when the large iron artifacts found in Stratum Ia (Omura 1998)° are included, indicates that iron or steel making and the manufacturing of iron artifacts underwent d

In [ ]:
def tokens_to_sentences(tokens):
    sentences = []
    current_sentence = []
    for token in tokens:
        current_sentence.append(token)
        if token.endswith('.'):  # Assuming '.' marks the end of a sentence
            sentences.append(' '.join(current_sentence))
            current_sentence = []
    # Add the last sentence if it doesn't end with a period
    if current_sentence:
        sentences.append(' '.join(current_sentence))
    return sentences
sentences = tokens_to_sentences(data)

In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Process the sentence
docs = [nlp(sentence) for sentence in sentences]

# Iterate over processed docs
for doc in docs:
    for token in doc:
        print(f"{token.text} -> {token.dep_} -> {token.head.text}")
    print("\n---\n")  # Print a separator between sentences

﻿IRON -> csubj -> REMAINS
OBJECTS -> dobj -> ﻿IRON
FROM -> prep -> ﻿IRON
THE -> det -> ARCHITECTURAL
ARCHITECTURAL -> pobj -> FROM
REMAINS -> ROOT -> REMAINS
OF -> prep -> REMAINS
STRATUM -> compound -> III
III -> pobj -> OF
AND -> cc -> III
STRATUM -> compound -> II
II -> conj -> III
AT -> nmod -> CORRELATION
KAMAN -> compound -> KALEHOYUK
- -> punct -> KALEHOYUK
KALEHOYUK -> pobj -> AT
: -> punct -> CORRELATION
CORRELATION -> ROOT -> CORRELATION
BETWEEN -> prep -> CORRELATION
COMPOSITION -> nmod -> LEVELS
AND -> cc -> COMPOSITION
ARCHAEOLOGICAL -> conj -> COMPOSITION
LEVELS -> nmod -> AKANUMA
Hideo -> compound -> AKANUMA
AKANUMA -> pobj -> BETWEEN
1 -> nummod -> INTRODUCTION
INTRODUCTION -> npadvmod -> CORRELATION
During -> prep -> find
excavation -> pobj -> During
from -> prep -> excavation
1986 -> pobj -> from
to -> prep -> from
2001 -> pobj -> to
at -> prep -> from
the -> det -> site
site -> pobj -> at
of -> prep -> site
Kaman -> compound -> Kalehdyiik
- -> punct -> Kalehdyiik
Kal